In [150]:
import panel as pn
import param as pm
import pandas as pd
import numpy as np
from glob import glob
from bokeh.palettes import Category20, Turbo256
import hvplot.pandas
hvplot.extension('bokeh')
pn.extension()
# pd.set_option('display.max_rows', 5)

# Assuming other imports and setup are done as before

class Simulation(pm.Parameterized):
    experiment = pm.ObjectSelector(default='sanity_check_run', objects=list(set([f.split('/')[-1].split('-')[0] for f in glob('../data/simulations/*')])))
    dataset = pm.Selector(default=None)  # This will be dynamically populated
    drop_list = pm.List(precedence=-1, default=['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'])
    sim_df = pm.DataFrame(precedence=-1)
    max_rows = pm.Integer(6, bounds=(1, None), step=2)
    color_palette = pm.Selector(default=Category20, objects=[Category20, Turbo256], precedence=1)
    column_colors = pm.Dict(precedence=-1)
    value_format = pm.Selector(default='Scientific', objects=['Scientific', 'Millify', 'Decimal'])
    value_color = pm.Selector(default='Columns', objects=['Rows', 'Columns', 'None'])

    def __init__(self, **params):
        super(Simulation, self).__init__(**params)
        self._update_dataset_options()
        self._load_simulation_data()

    @pm.depends('experiment', watch=True)
    def _update_dataset_options(self, event=None):
        datasets = sorted(glob(f"../data/simulations/{self.experiment}*"))
        self.param.dataset.objects = datasets
        self.dataset = datasets[-1] if datasets else None

    @pm.depends('dataset', 'drop_list', watch=True)
    def _load_simulation_data(self):
        """Load the simulation data when dataset or drop_list are changed."""
        if self.dataset:
            self.sim_df = pd.read_pickle(self.dataset).drop(self.drop_list, axis=1)
        else:
            self.sim_df = pd.DataFrame()

    def _discrete_colorization(self):
        column_colors = {col: self.color_palette[20][i%20] for i, col in enumerate(self.sim_df.columns)}
        return column_colors

    def _continuous_colorization(self):
        column_colors = dict(zip(self.sim_df.columns, [self.color_palette[int(i)] for i in np.linspace(0,len(self.color_palette)-1, len(self.sim_df.columns))]))
        return column_colors
        
    @pm.depends('sim_df', 'color_palette', watch=True)
    def _update_column_colors(self):
        """Set column colors based on selected color palette and sim df"""
        if self.color_palette == Turbo256:
            self.column_colors = self._continuous_colorization()
        
        if self.color_palette == Category20:
            self.column_colors = self._discrete_colorization()

    def _truncate_dataframe(self, df):
        return pd.concat([df.head(self.max_rows//2), df.tail(self.max_rows//2)])
    

    def view_results_dataframe(self):
        header_styles = [{
            'selector': f'th.col_heading.level0.col{i}',
            'props': [('background-color', self.column_colors.get(col))]
        } for i, col in enumerate(self.sim_df.columns)]
        
        # df.columns.name = 'balance'
        
        # describe_difference_df_styled = df.style.map(color_scale).set_table_styles(header_styles)
        # describe_difference_df_styled
        # return pn.panel(self._truncate_dataframe(self.sim_df).style.set_table_styles(header_styles), max_rows=self.max_rows)
        return pn.panel(self._truncate_dataframe(self.sim_df).style.set_table_styles(header_styles), max_rows=self.max_rows)



    # def view_results_dataframe(self):
    #     # Assuming self.sim_df is the DataFrame you want to style
    #     truncated_df = self._truncate_dataframe(self.sim_df)
        
    #     # Generate a continuous color map based on Turbo palette for cell values
    #     def apply_turbo_color_scale(val, min_val, max_val):
    #         normalized = (val - min_val) / (max_val - min_val) if max_val > min_val else 0
    #         color_idx = int(normalized * (len(Turbo256) - 1))
    #         return f'background-color: {Turbo256[color_idx]}'
        
    #     # Apply the color scale to each cell
    #     def style_df(df):
    #         min_val, max_val = df.min().min(), df.max().max()
    #         styled = df.style.applymap(lambda val: apply_turbo_color_scale(val, min_val, max_val))
    #         return styled
        
    #     styled_df = style_df(truncated_df)
    #     return pn.panel(styled_df, max_rows=self.max_rows)

    # def view_results_dataframe(self):
    #     # Ensure only numeric columns are considered for color scaling
    #     numeric_df = self.sim_df.select_dtypes(include=np.number)
    #     truncated_numeric_df = self._truncate_dataframe(numeric_df)
        
    #     # Calculate min and max across numeric columns for scaling
    #     min_val = truncated_numeric_df.min().min()
    #     max_val = truncated_numeric_df.max().max()
    
    #     # Define the color scaling function using the Turbo palette
    #     def apply_turbo_color_scale(val):
    #         if pd.isnull(val):
    #             return 'background-color: #ffffff'  # Return white for NaN values
    #         # Prevent division by zero or no variation
    #         if max_val > min_val:
    #             # Normalize value
    #             normalized = (val - min_val) / (max_val - min_val)
    #         else:
    #             # Default to middle of the palette for no variation
    #             normalized = 0.5
    #         color_idx = int(normalized * (len(Turbo256) - 1))
    #         return f'background-color: {Turbo256[color_idx]}'

        
    #     # Apply color scale to each cell in numeric columns
    #     styled_df = truncated_numeric_df.style.map(apply_turbo_color_scale)
        
    #     # Return styled DataFrame as a Panel component
    #     return pn.panel(styled_df, max_rows=self.max_rows)



    def runs_overview(self):
        return self.sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='Days')

    def view_runs_overview(self):
        return pn.panel(self.runs_overview(), max_rows=self.max_rows)


    def view_color_columns_bar(self):
        """ View the colormap """

        # For some odd reason, hvplot reverses bar ordering when there are greater that 10 columns. So we apply a reverse to negate that hvplot bug. See here: https://github.com/holoviz/hvplot/issues/1277
        columns_reversed = self.sim_df.columns[::-1]
        
        return self.sim_df.count().to_frame().T.hvplot.bar(y=columns_reversed, color=[self.column_colors[c] for c in columns_reversed], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

    def view(self):
        """View the selected simulation results."""
        pd.set_option('display.max_rows', self.max_rows)
        view = pn.Column(
            """
            ## Simulation Analysis Dashboard
            """,
            pn.Accordion(
                ('Select Parameters', pn.Row(self, pn.Column('## Simulation Results DataFrame', self.view_results_dataframe))), 
                ('Runs Overview', self.view_runs_overview),
            ),
        )
        # view[1].active = list(range(len(view[1])))
        # view[1].active = []
        view[1].active = [0]
        return view


# Usage
s = Simulation()
df = s.sim_df
s.view()


Column
    [0] Markdown(str)
    [1] Accordion(active=[0])
        [0] Row
            [0] Column(margin=(5, 10), name='Simulation')
                [0] StaticText(value='<b>Simulation</b>')
                [1] Select(name='Experiment', options=OrderedDict([('sanity_chec...]), value='sanity_check_run')
                [2] Select(name='Dataset', options=OrderedDict([('../data/sim...]), value='../data/simulations/sanit...)
                [3] IntInput(name='Max rows', start=1, step=2, value=6)
                [4] Select(name='Value format', options=OrderedDict([('Scientific'...]), value='Scientific')
                [5] Select(name='Value color', options=OrderedDict([('Rows', ...]), value='Columns')
                [6] Select(name='Color palette', options=OrderedDict([("{3: ('#1f77...]), value={3: ('#1f77b4', ...})
            [1] Column
                [0] Markdown(str)
                [1] ParamMethod(method, _pane=DataFrame, defer_load=False)
        [1] ParamMethod(method, _pane=DataFrame, defer_load=False, name='Runs Overview')

In [143]:
s.view_color_columns_bar()

:Bars   [index,Variable]   (value)

In [91]:
s.view_color_columns_bar().opts(multi_level=False)

:Bars   [index,Variable]   (value)

In [125]:
s.sim_df.count().to_frame().reset_index().hvplot.bar(color=[s.column_colors[c] for c in s.sim_df.columns], rot=90, y='0', by='index')

:Bars   [index]   (0)

In [133]:
s.sim_df.count().to_frame().reset_index().hvplot.bar(x='index', y='0', by=)

:Bars   [index]   (0)

In [129]:
s.sim_df.count().to_frame().reset_index().T.hvplot.bar(x='index', y='0')

:Bars   [index]   (0)

In [105]:
s.sim_df.count().to_frame().T.hvplot.bar().opts(multi_level=False)

:Bars   [index,Variable]   (value)

In [110]:
s.sim_df.count().to_frame().hvplot.bar(y='0')

:Bars   [index]   (0)

In [107]:
s.sim_df.count().to_frame().hvplot.bar()

:Bars   [index]   (0)

In [ ]:
color="weight"

In [2]:
df = pd.DataFrame([range(20) for _ in range(20)])

In [3]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
18,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
19,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19


In [4]:
df.count().to_frame().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20


In [5]:
df.count().to_frame().T.hvplot.bar(rot=90, width=1400, height=500)

:Bars   [index,Variable]   (value)

In [66]:
import pandas as pd
import hvplot.pandas

#### DF 10 or less

In [67]:
df = pd.DataFrame({col: range(20) for col in range(10)})

In [68]:
df.count().to_frame().T

,0,1,2,3,4,5,6,7,8,9
0,20,20,20,20,20,20,20,20,20,20


In [69]:
df.count().to_frame().T.hvplot.bar(rot=90, width=1400, height=100)

:Bars   [index,Variable]   (value)

In [70]:
df.count()[::-1].to_frame().T.hvplot.bar(rot=90, width=1400, height=100)

:Bars   [index,Variable]   (value)

#### DF greater than 10

In [71]:
df = pd.DataFrame({col: range(20) for col in range(11)})

In [72]:
df.count().to_frame().T

,0,1,2,3,4,5,6,7,8,9,10
0,20,20,20,20,20,20,20,20,20,20,20


In [73]:
df.count().to_frame().T.hvplot.bar(rot=90, width=1400, height=100)

:Bars   [index,Variable]   (value)

In [74]:
df.count()[::-1].to_frame().T.hvplot.bar(rot=90, width=1400, height=100)

:Bars   [index,Variable]   (value)

In [36]:
df.count()

0     20
1     20
2     20
      ..
8     20
9     20
10    20
Length: 11, dtype: int64

In [37]:
df = pd.DataFrame({col: range(20) for col in range(20)})

In [38]:
df.count().to_frame().T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20


In [39]:
df.count()[::-1].to_frame().T.hvplot.bar(rot=90, width=1400, height=500)

:Bars   [index,Variable]   (value)

In [16]:
s.view_color_columns_bar()

:Bars   [index,Variable]   (value)

1. Applying Color
2. Selecting Columns
3. Selecting Column Groups
4. Descriptors

In [17]:
s.column_colors

{'days_passed': '#1f77b4',
 'blocks_passed': '#aec7e8',
 'circulating_supply': '#ff7f0e',
 'user_supply': '#ffbb78',
 'earned_supply': '#2ca02c',
 'issued_supply': '#98df8a',
 'earned_minus_burned_supply': '#d62728',
 'total_supply': '#ff9896',
 'sum_of_stocks': '#9467bd',
 'storage_fee_per_rewards': '#c5b0d5',
 'block_utilization': '#8c564b',
 'dsf_relative_disbursal_per_day': '#c49c94',
 'reward_issuance_balance': '#e377c2',
 'other_issuance_balance': '#f7b6d2',
 'operators_balance': '#7f7f7f',
 'nominators_balance': '#c7c7c7',
 'holders_balance': '#bcbd22',
 'farmers_balance': '#dbdb8d',
 'staking_pool_balance': '#17becf',
 'fund_balance': '#9edae5',
 'burnt_balance': '#1f77b4',
 'nominator_pool_shares': '#aec7e8',
 'operator_pool_shares': '#ff7f0e',
 'block_reward': '#ffbb78',
 'blockchain_history_size': '#2ca02c',
 'total_space_pledged': '#98df8a',
 'allocated_tokens': '#d62728',
 'buffer_size': '#ff9896',
 'reference_subsidy': '#9467bd',
 'average_priority_fee': '#c5b0d5',
 'aver

In [18]:
df = s.sim_df

In [19]:
df['blockchain_history_size'] / df['total_space_pledged']

0            NaN
14      1.295977
28      1.295977
          ...   
2548    1.296053
2562    1.296052
2576    1.296053
Length: 185, dtype: float64

In [20]:
df[['blockchain_history_size', 'total_space_pledged']]

,blockchain_history_size,total_space_pledged
0,0,0
14,2038498852864,1572944000000
28,4076997705728,3145888000000
...,...,...
2548,371028534493184,286275808000000
2562,373067033346048,287848752000000
2576,375105800634368,289421696000000


In [21]:
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}

NameError: name 'sim_df' is not defined

In [ ]:
header_styles = [{
    'selector': f'th.col_heading.level0.col{i}',
    'props': [('background-color', column_colors.get(col))]
} for i, col in enumerate(df.columns)]

df.columns.name = 'balance'

describe_difference_df_styled = df.style.map(color_scale).set_table_styles(header_styles)
describe_difference_df_styled